Load API Keys

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

Load Contents from HLB Fixed Deposite Pages

In [135]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader([
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit.html?icp=hlb-en-all-footer-txt-fd",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/fixed-deposit-account.html", 
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/e-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/senior-savers-flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/junior-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/foreign-fixed-deposit-account.html",
  "https://www.hlb.com.my/en/personal-banking/help-support/fees-and-charges/deposits.html",
])
data = loader.load()
data

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPERSONAL\n\n\nBUSINESS\n\n\n\n                            Priority\n\n\nPriority\nIslamic\nGlobal Markets\nBlue Header\nInterim Homepage\nPriority Banking Form\ntest\ntest\nLoan Cal\nLog Viewer\nasdsadsad\nstand alone page\nMaster Form Page\nMaster Form Page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n                            ENG\n\n\nLAN A\nLAN B\nLAN C\n\n\n\n\n                            Online Banking\n                        \n\nPersonal\nBusiness\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                About Us\n                            \n\nOur Story\nNews & Updates\nInvestor Relations\nCSR\nAwards\nManagement & BODs\nCareer\n\n\n\n\n                                Discover Our Products\n      

Split the Loaded Data into Chunks

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(data)
texts

[Document(page_content='Fixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPERSONAL\n\n\nBUSINESS\n\n\n\n                            Priority\n\n\nPriority\nIslamic\nGlobal Markets\nBlue Header\nInterim Homepage\nPriority Banking Form\ntest\ntest\nLoan Cal\nLog Viewer\nasdsadsad\nstand alone page\nMaster Form Page\nMaster Form Page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n                            ENG\n\n\nLAN A\nLAN B\nLAN C\n\n\n\n\n                            Online Banking\n                        \n\nPersonal\nBusiness\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                About Us\n                            \n\nOur Story\nNews & Updates\nInvestor Relations\nCSR\nAwards\nManagement & BODs\nCareer\n\n\n\n\n                                Discover Our Products\n                            \n\nDeposits\nCredit Card\nLoans\nInsurance\nInvestment\nRemittance\

Initialize Pinecone

In [5]:
import pinecone

pinecone.init(
  api_key=os.getenv("PINECONE_API_KEY"),
  environment=os.getenv("PINECONE_ENVIRONMENT")
)
index_name = "main-index"
namespace = "hlb-fixed-deposit-with-agent"
if index_name not in pinecone.list_indexes():
  pinecone.create_index(
    name=index_name,
    metric="cosine",
    dimension=1536
  )
index = pinecone.Index(index_name)
index.delete(delete_all=True, namespace=namespace)

d:\Code\langchain-test\venv\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


{}

Create Embeddings and Vectors to Insert to Pinecone Database

In [6]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name, namespace=namespace)

Create Retriever for Vector Search

In [125]:
retriever = vector.as_retriever(search_type="similarity_score_threshold" ,search_kwargs={'score_threshold': 0.6, 'k': 4})
retriever

VectorStoreRetriever(tags=['Pinecone', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x000001EBA1BD69D0>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.6, 'k': 4})

Creating Prompt Template

In [123]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate

llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
system_message = """
  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You should not answer anything other than provided document.
  If you don't have enought information from the document, don't answer with your general knowledge.
  Your answer should be detailed but if you have to do calculation, don't show your work or equations, double check your answer, just provide the final value, and you don't need to show reference.
  If the bank's name is not provided in the input, answer with the assumption that the question is about HLB.
  If the bank's name is provided and it's not HLB, don't answer that question.
  You don't need to answer if the input requires you to do something creative such as writing a song or making a joke.
"""
prompt = ChatPromptTemplate.from_messages([
  ("system", system_message),
  MessagesPlaceholder("chat_history", optional=True),
  ("human", "{input}"),
  MessagesPlaceholder("agent_scratchpad")
])

Creating Chain

In [126]:
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(memory_key="chat_history", input_key="input", k=3)

chain = RetrievalQA.from_chain_type(
  llm=llm,
  chain_type="stuff",
  retriever=retriever,
  verbose=True,
)

Creating Tool from Chain

In [127]:
from langchain.tools import Tool
from langchain.agents import load_tools

knowledge_base_tool = Tool(
  name="hlb_fixed_deposit_tool",
  func=chain,
  description=("This tool is to collect and reterieve information about fixed deposit for HongLeong Bank")
)

tools = [knowledge_base_tool]

Creating Agent

In [128]:
from langchain.agents import create_react_agent, AgentExecutor, initialize_agent,AgentType
from langchain.callbacks import get_openai_callback

# agent = create_openai_functions_agent(llm=llm, tools=[knowledge_base_tool], prompt=prompt)
# agent_executor = AgentExecutor(agent=agent, tools=[knowledge_base_tool], max_iterations=3, return_source_documents=True, verbose=True)
agent = initialize_agent(
  agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
  tools=tools,
  llm=llm,
  return_source_documents=True,
  verbose=True
)

def ask_question(question):
  with get_openai_callback() as cb: 
    # result = agent_executor.invoke({"input": question})
    result = agent.invoke(question)
    print(f'{cb}\n')
    print(result['output'])

Running the Chain

In [133]:
ask_question("What is HLB e-fixed deposit?")



> Entering new AgentExecutor chain...
To answer the question, I need to gather information about the HLB e-fixed deposit product.
Action: hlb_fixed_deposit_tool
Action Input: What is HLB e-fixed deposit?

> Entering new RetrievalQA chain...

> Finished chain.

Observation: {'query': 'What is HLB e-fixed deposit?', 'result': "HLB e-Fixed Deposit refers to the electronic Fixed Deposit service offered by Hong Leong Bank (HLB), which allows customers to place fixed deposits and manage them online through HLB Connect, the bank's internet banking platform. Customers can make new placements, view account and placement details, and withdraw funds anytime via HLB Connect. The service offers the convenience of managing fixed deposits remotely, without the need to visit a physical branch.\n\nKey features of the HLB e-Fixed Deposit include:\n\n- New placements and withdrawals can be made anytime via HLB Connect.\n- Account and placement details can be viewed upon successful placement.\n- Custome

In [129]:
ask_question("How many types of fixed deposit are there?")



> Entering new AgentExecutor chain...
To find out the number of fixed deposit types offered by HongLeong Bank, I will use the hlb_fixed_deposit_tool to retrieve this information.

Action: hlb_fixed_deposit_tool
Action Input: What are the types of fixed deposits available?


> Entering new RetrievalQA chain...

> Finished chain.

Observation: {'query': 'What are the types of fixed deposits available?\n', 'result': 'The types of fixed deposits available are:\n\n1. Fixed Deposit - Earn attractive interest with fixed tenures of up to 60 months.\n2. e-Fixed Deposit - Place your e-Fixed Deposit online via HLB Connect.\n3. Flexi Fixed Deposit - Enjoy the flexibility of making early partial withdrawals.\n4. Senior Savers Flexi Fixed Deposit - An attractive interest FD account for individuals aged 50 years and above.\n5. Junior Fixed Deposit - An attractive interest FD account for children below the age of 18.\n6. Foreign Currency Fixed Deposit - A convenient way to invest in foreign currency

In [ ]:
ask_question("What are the pros and cons of fixed deposit?")

In [130]:
ask_question("Who is Jay Z?")



> Entering new AgentExecutor chain...
Jay Z is not related to fixed deposit information for HongLeong Bank, so the hlb_fixed_deposit_tool would not be useful in answering this question.
Thought: I know the answer based on general knowledge.
Final Answer: Jay Z, whose real name is Shawn Corey Carter, is an American rapper, songwriter, record executive, businessman, and media proprietor. He is widely regarded as one of the most influential hip-hop artists in history.

> Finished chain.
Tokens Used: 247
	Prompt Tokens: 159
	Completion Tokens: 88
Successful Requests: 1
Total Cost (USD): $0.004229999999999999

Jay Z, whose real name is Shawn Corey Carter, is an American rapper, songwriter, record executive, businessman, and media proprietor. He is widely regarded as one of the most influential hip-hop artists in history.


In [ ]:
ask_question("Are you sure you dont know Jay Z? He is a very popular hip hop artist?")

In [131]:
ask_question("How many types of fixed deposit CIMB provide?")



> Entering new AgentExecutor chain...
I do not have access to a tool for CIMB fixed deposit information, and my knowledge is limited to what was available up until my last update in 2023. Therefore, I cannot provide the current number of fixed deposit types that CIMB provides.

Final Answer: I am unable to provide the current number of fixed deposit types that CIMB provides as I do not have access to a specific tool for CIMB and my knowledge is not up-to-date beyond 2023. It would be best to check directly with CIMB Bank for the most current information.

> Finished chain.
Tokens Used: 277
	Prompt Tokens: 164
	Completion Tokens: 113
Successful Requests: 1
Total Cost (USD): $0.00503

I am unable to provide the current number of fixed deposit types that CIMB provides as I do not have access to a specific tool for CIMB and my knowledge is not up-to-date beyond 2023. It would be best to check directly with CIMB Bank for the most current information.


In [32]:
ask_question("What are the interest rates for Fixed Deposit?")



> Entering new AgentExecutor chain...

Invoking: `hlb_fixed_deposit_tool` with `interest rates`


Based on the provided context, here are the interest rates for various products and services:

**BizOne Overdraft Facility (BizOne OD) Related:**
- Prescribed Rate (Interest payable on utilisation of OD): 1% p.a. above BLR on daily rest on the utilised OD amount.
- Excess Rate (Interest payable on overdrawn OD limit): 4% p.a. above BLR on daily rest on the amount drawn in excess of the approved OD limit.
- Commitment Fee (Interest payable on unutilised OD in excess of RM250,000.00): 1% p.a. on daily rest on the unutilised OD amount.

**e-Fixed Deposit (eFD) Rates (effective from 21/07/2023):**
- 1 month: 1.85% p.a.
- 2 months: 2.25% p.a.
- 3 months: 2.25% p.a.
- 4 months: 2.30% p.a.
- 5 months: 2.30% p.a.
- 6 months: 2.30% p.a.
- 7 months: 2.35% p.a.
- 8 months: 2.35% p.a.
- 9 months: 2.35% p.a.
- 10 months: 2.35% p.a.
- 11 months: 2.35% p.a.
- 12 months: 2.35% p.a.
- 13 - 60 months: 2.3

In [ ]:
ask_question("Can you find out by browsing the web?")

In [25]:
ask_question("Lets say I want to invest RM 5000 in Fixed Deposit for 36 months and the interest rate is 5%. Please calculate total withdrawal amount at the end of the term. Take note the interest rate is compund interest?")



> Entering new AgentExecutor chain...

Invoking: `hlb_fixed_deposit_tool` with `5000`


I'm sorry, but it seems like there is some information missing from your question. If you are asking about depositing RM5,000 into a Flexi FD or a similar account, please provide more context or clarify your question so I can assist you better. If you're inquiring about the interest you would earn on a RM5,000 deposit, please note that the provided interest rates apply to different tenures, and the minimum amount to open a Senior Savers Flexi FD is RM10,000. If you have a different question or need further assistance, feel free to ask!I'm sorry, but I cannot provide the total withdrawal amount for the scenario you described as the information provided does not match the available details for HLB's fixed deposit products. The interest rates and terms for SGD Fixed Deposits are not specified in the document I have access to. If you have more specific information about HLB's SGD Fixed Deposit rates a

In [35]:
ask_question("Lets say I want to invest RM 5000 in Fixed Deposit for 9 months. Please calculate total withdrawal amount at the end of the term.")



> Entering new AgentExecutor chain...

Invoking: `hlb_fixed_deposit_tool` with `5000`


I'm sorry, but it seems like there might be some missing context to your query. If you're referring to a financial product or service, could you please provide more details or clarify your question? If you're asking about the minimum amount required for a specific account or investment, please specify which product you are inquiring about.To calculate the total withdrawal amount for a RM 5000 Fixed Deposit for 9 months, I need to know the interest rate for that specific term. Could you please provide the interest rate for a 9-month Fixed Deposit?

> Finished chain.
Tokens Used: 1428
	Prompt Tokens: 1295
	Completion Tokens: 133
Successful Requests: 3
Total Cost (USD): $0.01694

To calculate the total withdrawal amount for a RM 5000 Fixed Deposit for 9 months, I need to know the interest rate for that specific term. Could you please provide the interest rate for a 9-month Fixed Deposit?


In [ ]:
ask_question("Can a non-resident of Malaysia invest in CIMB SGD fixed deposit?")

In [ ]:
ask_question("Who is the CEO of CIMB bank?")

In [ ]:
ask_question("What is CIMB net profile for the last financial year?")

In [ ]:
ask_question("What if Jay Z from United States wants to invest in SGD fixed deposit. Is that ok ?")

In [ ]:
ask_question("What if Michael Jackson wants to invest in SGD Fixed Deposit. Is that ok?")

In [26]:
ask_question("Write a poem for HLB fixed deposit starting with Roses are Red, Violets are blue")



> Entering new AgentExecutor chain...
I'm sorry, but I can't assist with writing poems or any other creative content. If you have any questions or need information related to HongLeong Bank's fixed deposit services, feel free to ask!

> Finished chain.
Tokens Used: 309
	Prompt Tokens: 266
	Completion Tokens: 43
Successful Requests: 1
Total Cost (USD): $0.00395

I'm sorry, but I can't assist with writing poems or any other creative content. If you have any questions or need information related to HongLeong Bank's fixed deposit services, feel free to ask!


In [ ]:
ask_question("how many types of saving accounts are there in a general bank except HLB?")

In [ ]:
ask_question("who is obama's bank?")